In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# Initialise paths
input_data_location = "https://github.com/AlexKitov/fastAPI/raw/main/covidServer/app/data/covid_testing_data.csv"

# Used when working from one drive
# project_location = '/content/drive/My Drive/NZ_TASK1'
# input_data_location = project_location + '/covid_testing_data.csv'
# output_data_location = project_location + '/monthly_covid_testing_data.csv'

In [3]:
# Initialise constants
COUNTRIES = ["Denmark", "Germany", "Romania", "Spain", "Sweden"]
LEVEL = "national"
YEAR = 2020

In [4]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime

# Used when working from one drive
# print(os.listdir(project_location))

In [5]:
df_in = pd.read_csv(input_data_location)
df_in

,country,country_code,year_week,level,region,region_name,new_cases,tests_done,population,testing_rate,positivity_rate,testing_data_source
0,Austria,AT,2020-W15,national,AT,Austria,1954,12339,8858775.0,139.285624,15.835967,Manual webscraping
1,Austria,AT,2020-W16,national,AT,Austria,773,58488,8858775.0,660.226724,1.321639,Manual webscraping
2,Austria,AT,2020-W17,national,AT,Austria,479,33443,8858775.0,377.512692,1.432288,Manual webscraping
3,Austria,AT,2020-W18,national,AT,Austria,349,26598,8858775.0,300.244673,1.312129,Country website
4,Austria,AT,2020-W19,national,AT,Austria,249,42153,8858775.0,475.833284,0.590705,Country website
...,...,...,...,...,...,...,...,...,...,...,...,...
3278,Sweden,SE,2020-W50,national,SE,Sweden,43639,270944,10230185.0,2648.476054,16.106280,TESSy
3279,Sweden,SE,2020-W51,national,SE,Sweden,46195,299447,10230185.0,2927.092716,15.426770,TESSy
3280,Sweden,SE,2020-W52,national,SE,Sweden,37147,232114,10230185.0,2268.913026,16.003774,TESSy
3281,Sweden,SE,2020-W53,national,SE,Sweden,41343,201011,10230185.0,1964.881378,20.567531,TESSy


# Data wrangling

https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html


## Initial claning and filtering

After initial look at the data the plan is to:
1. Select only `requested countries`
2. Select only `national` data
3. Add a column of type `date` based on the `year_week` column where Monday of each week is chosen to determine in which month the data belongs to
4. Create `month` column for visualisation
5. Select only data from year 2020, based on the `date` column

In [ ]:
to_year_month = lambda year_week: datetime.strptime(year_week + '-1', "%G-W%V-%w")
to_month_number = lambda year_month: year_month.month
to_month_name = lambda year_month: year_month.strftime("%b")
def is_equal(year): return lambda date: date.year == year

In [ ]:
# Initial Data cleaning 
df = df_in \
      .query(f'country in {COUNTRIES}') \
      .query(f'level=="{LEVEL}"')

df.drop(['level', 'region', 'region_name'], axis='columns', inplace=True)

# Preparation for aggregaition
df["year_month"] = df.year_week.apply(to_year_month)
df["month_num"] = df.year_month.apply(to_month_number)
df["month"] = df.year_month.apply(to_month_name)
df = df[df.year_month.apply(is_equal(YEAR))]

df.head()

,country,country_code,year_week,new_cases,tests_done,population,testing_rate,positivity_rate,testing_data_source,year_month,month_num,month
385,Denmark,DK,2020-W05,0,3,5806081.0,0.051670,0.000000,Country website,2020-01-27,1,Jan
386,Denmark,DK,2020-W06,0,4,5806081.0,0.068893,0.000000,Country website,2020-02-03,2,Feb
387,Denmark,DK,2020-W07,0,4,5806081.0,0.068893,0.000000,Country website,2020-02-10,2,Feb
388,Denmark,DK,2020-W08,0,9,5806081.0,0.155010,0.000000,Country website,2020-02-17,2,Feb
389,Denmark,DK,2020-W09,4,337,5806081.0,5.804259,1.186944,Country website,2020-02-24,2,Feb


## Aggregation

The apprioach is:
1. `groupby` constants
2. sum the numerical values per `country` and `month`
3. Sort by `country` and `month`
4. Correct the `positivity_rate` value as it cannot be simply summed
5. Correct data problems where `positivity_rate` gets over `100%`

In [ ]:
# Aggregation
df = df.groupby(['country', 'month_num', 'month', 'population', 'country_code']) \
       .sum() \
       .reset_index() \
       .sort_values(["country", "month_num"])

# Posotovoty rate needs to be recalculated
df["positivity_rate"] = df.new_cases / df.tests_done * 100

# month_num was only needed for sorting
df.drop(["month_num"], axis='columns', inplace=True)

df.head()

,country,month,population,country_code,new_cases,tests_done,testing_rate,positivity_rate
0,Denmark,Jan,5806081.0,DK,0,3,0.051670,0.000000
1,Denmark,Feb,5806081.0,DK,4,354,6.097056,1.129944
2,Denmark,Mar,5806081.0,DK,4365,51405,885.364844,8.491392
3,Denmark,Apr,5806081.0,DK,5154,234516,4039.144476,2.197718
4,Denmark,May,5806081.0,DK,2146,343774,5920.930142,0.624247


In [ ]:
# Fix data problem with Romania
df.positivity_rate = df.positivity_rate.apply(lambda rate: None if rate > 100 else rate)

dd = df[df.country=="Romania"]
dd.head(11)

,country,month,population,country_code,new_cases,tests_done,testing_rate,positivity_rate
22,Romania,Mar,19414458.0,RO,1621,14,0.072111,NaN
23,Romania,Apr,19414458.0,RO,9299,150753,776.498628,6.168368
24,Romania,May,19414458.0,RO,6094,232964,1199.951088,2.615855
25,Romania,Jun,19414458.0,RO,9716,324197,1669.874070,2.996943
26,Romania,Jul,19414458.0,RO,24213,428557,2207.411610,5.649890
27,Romania,Aug,19414458.0,RO,41828,669775,3449.877406,6.245082
28,Romania,Sep,19414458.0,RO,39051,567467,2922.909308,6.881634
29,Romania,Oct,19414458.0,RO,112598,743541,3829.831356,15.143482
30,Romania,Nov,19414458.0,RO,266913,977899,5036.962659,27.294537
31,Romania,Dec,19414458.0,RO,126853,452092,2328.635700,28.059112


In [ ]:
dd[["month", "tests_done"]]

,month,tests_done
22,Mar,14
23,Apr,150753
24,May,232964
25,Jun,324197
26,Jul,428557
27,Aug,669775
28,Sep,567467
29,Oct,743541
30,Nov,977899
31,Dec,452092


In [ ]:
# Used when working from one drive
# df.to_csv(output_data_location, index=False)

# Visualisation

https://plotly.com/python/facet-plots/

In [ ]:
import plotly.express as px

def plotly_line(y):
  fig = px.line(df, 
                x="month", y=y, facet_col="country", 
                facet_col_wrap=7,
                # facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
                # facet_col_spacing=0.04, # default is 0.03
                height=500, 
                width=1400,
                title=f"Evolution of '{y}' COVID-19 cases for {YEAR}")

  fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
  fig.update_yaxes(showticklabels=True)
  return fig

In [ ]:
plotly_line("new_cases").show()

In [ ]:
plotly_line("testing_rate").show()

In [ ]:
plotly_line("positivity_rate").show()